<a href="https://colab.research.google.com/github/eze-hong/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EC%B6%9C%EC%84%9D%EA%B3%BC%EC%A0%9C_%ED%99%8D%EC%9D%B4%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-2

1.   Python을 사용하여 API를 호출하는 코드를 작성
2.   2015년 1월부터 2024년 12월까지의 개인 유형의 현년 전기, 가스, 수도, 지역난방 에너지 사용량 데이터를 수집하는 프로그램을 작성
3. API 호출 성공을 확인할 수 있는 실행 결과를 캡처하여 첨부


In [1]:
import requests
import json
import pandas as pd
import numpy as np

url = 'http://openapi.seoul.go.kr:8088/626b6c4a5873656f36336e756f4b6f/json/energyUseDataSummaryInfo/1/5/'

params = {
    'KEY' : '626b6c4a5873656f36336e756f4b6f',
    'TYPE' : 'json',
    'SERVICE': 'energyUseDataSummaryInfo',
    'START_INDEX' : '1',
    'END_INDEX' : '100',
}
## 데이터 수집
response = requests.get(url, params=params)

data = response.json()['energyUseDataSummaryInfo'] ##['row']
##df = pd.DataFrame(data)
print(data)
##print(data['RESULT']['CODE'])

{'list_total_count': 1241, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2025', 'MON': '06', 'MM_TYPE': '개인', 'CNT': '1269454', 'EUS': '278745288', 'EUS1': '275851011', 'EUS2': '273151435', 'ECO2_1': '4244065', 'ECO2_2': '2076623.656', 'GUS': '13084768', 'GUS1': '11889039', 'GUS2': '12094650', 'GCO2_1': '1091102', 'GCO2_2': '2182204', 'WUS': '14927683.6', 'WUS1': '15166284.8', 'WUS2': '15371662.2', 'WCO2_1': '-341289.9', 'WCO2_2': '-611822.446', 'HUS': '379182.22', 'HUS1': '368839.049', 'HUS2': '389847.167', 'HCO2_1': '-160.888', 'HCO2_2': '-22448.67848156', 'REG_DATE': '2025-09-01 04:02:15.0'}, {'YEAR': '2025', 'MON': '06', 'MM_TYPE': '학교', 'CNT': '6745', 'EUS': '117808793', 'EUS1': '114793127', 'EUS2': '114623246', 'ECO2_1': '3100606.5', 'ECO2_2': '3096973.668', 'GUS': '2093441', 'GUS1': '2046804', 'GUS2': '2061648', 'GCO2_1': '39207', 'GCO2_2': '78414', 'WUS': '1824338.2', 'WUS1': '1914804.2', 'WUS2': '1956560.2', 'WCO2_1': '-111344', 'WCO2_2': '-333633.

In [13]:
import requests
import json
import pandas as pd
import numpy as np

url = 'http://openapi.seoul.go.kr:8088/626b6c4a5873656f36336e756f4b6f/json/energyUseDataSummaryInfo/1/5/'

params = {
    'KEY' : '626b6c4a5873656f36336e756f4b6f',
    'TYPE' : 'json',
    'SERVICE': 'energyUseDataSummaryInfo',
    'START_INDEX' : '1',
    'END_INDEX' : '100',
}
## 데이터 수집
##response = requests.get(url, params=params)

##data = response.json()['energyUseDataSummaryInfo']['row']
##print(json.dumps(data, indent=4, ensure_ascii=False))

##df = pd.DataFrame(data)
##print(df)

##개인타입인지 확인
##is_개인 = df['MM_TYPE'] == '개인'

##확인한 결과로 필터링한 데이터
##개인df = df[is_개인]
##print(개인df)
##EUS전기
##selected_col = 개인df[['YEAR']['MON']['MM_TYPE']['EUS']['GUS']['WUS']['HUS']]
##selected_col = 개인df[['YEAR','MON','MM_TYPE','EUS','GUS','WUS','HUS']]
##print(selected_col)


def get_year_months():
  year_months = []
  for year in range(2015, 2016+1):
    ##for year in range(2015, 2024+1):
    for month in range(1,12+1):
      year_months.append(f'{year}{month:02d}')
  return year_months

responses = []

for year_month in get_year_months():
    year = year_month[:4]
    mon = year_month[4:]
    url = f'http://openapi.seoul.go.kr:8088/626b6c4a5873656f36336e756f4b6f/json/energyUseDataSummaryInfo/1/5/{year}/{mon}'
    response = requests.get(url, params=params)
    data = response.json()['energyUseDataSummaryInfo']
    if data['RESULT']['CODE'] == 'INFO-000' :
        rows =  data['row']
        개인_row = [r for r in rows if r['MM_TYPE'] == '개인']
        if(len(개인_row)!=0) :
            ##print([{'YEAR': d['YEAR'], 'MON' : d['MON'], 'MM_TYPE' : d['MM_TYPE'], 'EUS' : d['EUS'], 'GUS' : d['GUS'], 'WUS' : d['WUS'], 'HUS' : d['HUS']} for d in 개인_row])
            responses.append([{'YEAR': d['YEAR'], 'MON' : d['MON'], 'MM_TYPE' : d['MM_TYPE'], 'EUS' : d['EUS'], 'GUS' : d['GUS'], 'WUS' : d['WUS'], 'HUS' : d['HUS']} for d in 개인_row])

print(responses)
df = pd.DataFrame(response)
##df['YEAR','MON','MM_TYPE','EUS','GUS','WUS','HUS']

if(len(responses)!=0):
    print("api 호출 성공")
else :
    print("api 호출 실패")

[[{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'EUS': '193784708', 'GUS': '59133720', 'WUS': '12819757.886', 'HUS': '22740838.937'}], [{'YEAR': '2015', 'MON': '02', 'MM_TYPE': '개인', 'EUS': '189974230', 'GUS': '56487358', 'WUS': '12656888.218', 'HUS': '18793320.213'}], [{'YEAR': '2015', 'MON': '05', 'MM_TYPE': '개인', 'EUS': '177289769', 'GUS': '22060892', 'WUS': '13086734.214', 'HUS': '1320895.801'}], [{'YEAR': '2015', 'MON': '06', 'MM_TYPE': '개인', 'EUS': '186073153', 'GUS': '12677645', 'WUS': '14241651.402', 'HUS': '294312.634'}], [{'YEAR': '2015', 'MON': '07', 'MM_TYPE': '개인', 'EUS': '165114568', 'GUS': '8893640', 'WUS': '12083879.038', 'HUS': '702863.645'}], [{'YEAR': '2015', 'MON': '08', 'MM_TYPE': '개인', 'EUS': '242000479', 'GUS': '7619018', 'WUS': '15164553.742', 'HUS': '460288.965'}], [{'YEAR': '2015', 'MON': '10', 'MM_TYPE': '개인', 'EUS': '186473958', 'GUS': '8924018', 'WUS': '14711828.766', 'HUS': '3167520.127'}], [{'YEAR': '2015', 'MON': '12', 'MM_TYPE': '개인', 'EUS': '203388142

In [10]:
original_list = [
    {'name': 'Alice', 'age': 30, 'city': 'New York'},
    {'name': 'Bob', 'age': 24, 'city': 'London'},
    {'name': 'Charlie', 'age': 35, 'city': 'Paris'}
]

# Extract 'name' and 'city'
new_list = [{'name': d['name'], 'city': d['city']} for d in original_list] ##this called 'List Comprehension'
print(new_list)

[{'name': 'Alice', 'city': 'New York'}, {'name': 'Bob', 'city': 'London'}, {'name': 'Charlie', 'city': 'Paris'}]


In [15]:
df

,0
0,"b'{""energyUseDataSummaryInfo"":{""list_total_cou..."
1,"b'""YEAR"":""2016"",""MON"":""12"",""MM_TYPE"":""\xec\xa2..."
2,"b'0.5"",""ECO2_2"":""-44817.012"",""GUS"":""327872"",""G..."
3,"b'US1"":""40326"",""WUS2"":""38880"",""WCO2_1"":""3695"",..."
4,"b'TE"":""2017-03-05 04:00:00.0""},{""YEAR"":""2016"",..."
5,"b'2"":""140354044"",""ECO2_1"":""864940.5"",""ECO2_2"":..."
6,"b'O2_2"":""360478.72"",""WUS"":""2300595.4"",""WUS1"":""..."
7,"b'HUS1"":""10400.538"",""HUS2"":""14403.696"",""HCO2_1..."
8,"b'16"",""MON"":""12"",""MM_TYPE"":""\xea\xb3\xb5\xeb\x..."
9,"b'-10761031.5"",""ECO2_2"":""-4562677.356"",""GUS"":""..."


문제 2
1. 수집한 JSON 형태의 데이터를 pandas DataFrame으로 변환하고, 데이터의 기본 정보를 출력 하는 코드와 실행 결과를 첨부
2. 연도별, 계절별 분석을 위해 날짜 컬럼을 활용하여 연도(year)와 계절(season) 컬럼을 추가하 는 전처리 코드를 작성하고, 변환 결과를 확인할 수 있는 출력 결과를 첨부하시오.
※ 계절 구분: 봄(3-5월), 여름(6-8월), 가을(9-11월), 겨울(12-2월)

In [19]:
import numpy as np
df = pd.DataFrame(responses)
df.info()
df.head()
print("::::::columns::::::")
print(df.columns)

print(df.values.tolist())

print(df.values.flatten())

df.describe(include='all')
##df.describe(include=[np.number])
##df['SEASON'] = df['MON'].apply() ##7강 참

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       17 non-null     object
dtypes: object(1)
memory usage: 268.0+ bytes
::::::columns::::::
RangeIndex(start=0, stop=1, step=1)
[[{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'EUS': '193784708', 'GUS': '59133720', 'WUS': '12819757.886', 'HUS': '22740838.937'}], [{'YEAR': '2015', 'MON': '02', 'MM_TYPE': '개인', 'EUS': '189974230', 'GUS': '56487358', 'WUS': '12656888.218', 'HUS': '18793320.213'}], [{'YEAR': '2015', 'MON': '05', 'MM_TYPE': '개인', 'EUS': '177289769', 'GUS': '22060892', 'WUS': '13086734.214', 'HUS': '1320895.801'}], [{'YEAR': '2015', 'MON': '06', 'MM_TYPE': '개인', 'EUS': '186073153', 'GUS': '12677645', 'WUS': '14241651.402', 'HUS': '294312.634'}], [{'YEAR': '2015', 'MON': '07', 'MM_TYPE': '개인', 'EUS': '165114568', 'GUS': '8893640', 'WUS': '12083879.038', 'HUS': '702863.645'}], [{'YEAR': '2015', '

ValueError: No objects to concatenate